# Bowling Stats

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob, yaml, pickle
import datetime

from utils import get_player_profile
from utils import add_overs, Over2Balls

In [2]:
df_test=pickle.load(open('./database/bowling_record_all_years.df', 'rb'))
df_test.head(2)

,bowler,O,M,R,W,0s,1s,2s,3s,4s,...,NB,Team,Against,Win,Tied,Toss,team-total,season,date,match-id
0,P Kumar,4,0,38,0,14,3,2,0,3,...,0,Royal Challengers Bangalore,Kolkata Knight Riders,False,False,True,222,2008,2008-04-18,335982
1,Z Khan,4,0,38,1,8,8,3,0,3,...,0,Royal Challengers Bangalore,Kolkata Knight Riders,False,False,True,222,2008,2008-04-18,335982


In [3]:
def bowling_summary():
    df=pickle.load(open('./database/bowling_record_all_years.df', 'rb'))
    all_players = df['bowler'].unique()
    
    data=[]
    for player in all_players:
        
        dfp = df[ df['bowler'] == player ]
        Inns_ = dfp.shape[0]
        
        Ovs_  = add_overs(dfp['O'].values)
        Wkts_ = dfp['W'].sum()
        Runs_ = dfp['R'].sum()

        #HS_   = max(dfp.Runs) to be calculated for best bowling figures
        
        Balls_ = Over2Balls(Ovs_)
        
        SR_   = 0.
        Ave_  = 0.
        if Wkts_ > 0:
            SR_   = round( Balls_ / Wkts_ , 2 )
            Ave_  = round( Runs_ / Wkts_  , 2 )
            
        Fours_= dfp['4s'].sum()
        Sixes_= dfp['6s'].sum()
        WDs_  = dfp['WD'].sum()
        NBs_  = dfp['NB'].sum()

        NoWs_= ((dfp['W']==0)).sum()
        ThreeWs_= ((dfp['W']>=3)).sum()
        FourWs_ = ((dfp['W']>=4)).sum()
        FiveWs_ = ((dfp['W']>=5)).sum()

        data.append([player, Inns_, Ovs_, Wkts_, Runs_, SR_, Ave_, Fours_, Sixes_,
                     WDs_, NBs_, NoWs_, ThreeWs_, FourWs_, FiveWs_])
    
    df_p = pd.DataFrame(data, columns=['Player', 'Innings', 'Overs', 'Wickets', 'Runs',
                                       'SR', 'Ave', 'Fours', 'Sixes', 'WDs', 'NBs',
                                       '0-Fers', '3-Fers', '4-Fers', '5-Fers']) 
    
    return df_p

dfs= bowling_summary()
dfs.head()

,Player,Innings,Overs,Wickets,Runs,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
0,P Kumar,119,420.4,90,3269,28.04,36.32,338,104,112,1,57,3,0,0
1,Z Khan,99,366.4,103,2798,21.36,27.17,312,64,74,2,35,10,1,0
2,AA Noffke,1,4,1,40,24.00,40.00,2,2,1,0,0,0,0,0
3,JH Kallis,89,290.2,65,2306,26.80,35.48,236,58,48,9,46,2,0,0
4,SB Joshi,4,9.1,1,82,55.00,82.00,6,5,0,0,3,0,0,0


## Top players in different categories

In [4]:
## Top players in terms of Runs, 50s, 100s, 4s, 6s and so ondef sorted_table(sort_by='Wickets', min_wkts=100):
def sorted_table(sort_by='Wickets', min_wkts=100, ascending=False):
    df_summary = bowling_summary()
    df_summary = df_summary[ (df_summary['Wickets']>=min_wkts) ]
    df_sorted  = df_summary.sort_values(by=[sort_by], ascending=ascending)
    return df_sorted

In [5]:
def get_top_players():
    sorting_list = ['Innings', 'Overs', 'Wickets', 'SR', 'Ave', '3-Fers', 'Fours', 'Sixes', 'WDs', 'NBs']
    for sort_item in sorting_list:
        print ('Sorted by :',sort_item)

        if sort_item in ['Fours', 'Sixes', 'WDs', 'NBs', 'SR', 'Ave']:
            df_sorted = sorted_table(sort_by=sort_item, ascending=True)
        else:
            df_sorted = sorted_table(sort_by=sort_item, ascending=False)

        display(df_sorted[:5])

get_top_players()

Sorted by : Innings


,Player,Innings,Overs,Wickets,Runs,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
27,PP Chawla,163,541.4,156,4280,20.83,27.44,316,181,33,2,59,12,2,0
49,Harbhajan Singh,157,562.2,150,3982,22.49,26.55,289,142,74,3,64,11,2,1
89,RA Jadeja,155,452.1,114,3471,23.80,30.45,213,148,44,2,85,11,4,1
144,R Ashwin,151,538.2,138,3708,23.41,26.87,226,135,95,2,56,7,1,0
84,A Mishra,150,526.5,160,3876,19.76,24.23,222,172,51,21,54,16,4,1


Sorted by : Overs


,Player,Innings,Overs,Wickets,Runs,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
49,Harbhajan Singh,157,562.2,150,3982,22.49,26.55,289,142,74,3,64,11,2,1
27,PP Chawla,163,541.4,156,4280,20.83,27.44,316,181,33,2,59,12,2,0
144,R Ashwin,151,538.2,138,3708,23.41,26.87,226,135,95,2,56,7,1,0
84,A Mishra,150,526.5,160,3876,19.76,24.23,222,172,51,21,54,16,4,1
103,SL Malinga,122,471.1,170,3399,16.63,19.99,314,86,129,18,31,19,7,1


Sorted by : Wickets


...

,Player,Innings,Overs,Wickets,Runs,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
1,Z Khan,99,366.4,103,2798,21.36,27.17,312,64,74,2,35,10,1,0
45,A Nehra,88,318,106,2507,18.00,23.65,279,76,57,9,26,14,1,0
251,JJ Bumrah,92,348.4,109,2599,19.19,23.84,232,79,45,22,33,14,2,0
103,SL Malinga,122,471.1,170,3399,16.63,19.99,314,86,129,18,31,19,7,1
211,B Kumar,121,449.3,136,3264,19.83,24.00,336,88,89,9,43,12,3,1


Sorted by : WDs


,Player,Innings,Overs,Wickets,Runs,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
27,PP Chawla,163,541.4,156,4280,20.83,27.44,316,181,33,2,59,12,2,0
231,SP Narine,119,464.1,127,3158,21.93,24.87,234,107,37,2,43,12,7,1
89,RA Jadeja,155,452.1,114,3471,23.80,30.45,213,148,44,2,85,11,4,1
251,JJ Bumrah,92,348.4,109,2599,19.19,23.84,232,79,45,22,33,14,2,0
43,R Vinay Kumar,104,353.3,105,2980,20.20,28.38,296,98,48,10,40,10,1,0


Sorted by : NBs


,Player,Innings,Overs,Wickets,Runs,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
1,Z Khan,99,366.4,103,2798,21.36,27.17,312,64,74,2,35,10,1,0
27,PP Chawla,163,541.4,156,4280,20.83,27.44,316,181,33,2,59,12,2,0
89,RA Jadeja,155,452.1,114,3471,23.80,30.45,213,148,44,2,85,11,4,1
144,R Ashwin,151,538.2,138,3708,23.41,26.87,226,135,95,2,56,7,1,0
231,SP Narine,119,464.1,127,3158,21.93,24.87,234,107,37,2,43,12,7,1
